In [1]:
import torch
from torch import optim, nn
from torch.utils.data import DataLoader
import torchvision
from torchvision import transforms
from lenet5 import Lenet5
from lenet5_withactivation import Lenet5 as Lenet5_wa
#from spatter_dataset_scores import Spatter
from spatter_dataset import Spatter_gray
import visdom
from tqdm import tqdm
import pandas as pd
import numpy as np
import random

# for better repeatability
manualSeed = 1234
np.random.seed(manualSeed)
random.seed(manualSeed)
torch.manual_seed(manualSeed)

In [2]:
# device = torch.device('cuda')
device = torch.device('cpu')

batch_size = 1024
lr = 1e-3
epochs = 20

# 6 files, 2 labels
root = 'data'
file_list = ['left_low_0.h5', 'left_high_2.h5',
             'mid_low_0.h5', 'mid_high_2.h5',
             'right_low_0.h5', 'right_high_2.h5']
label_list = [0, 1, 0, 1, 0, 1]  # 2 classes={0:low gas flow, 1: high gas flow}

train_db = Spatter_gray(file_list, label_list, resize=32, mode='train', root=root)
val_db = Spatter_gray(file_list, label_list, resize=32, mode='val', root=root)
test_db = Spatter_gray(file_list, label_list, resize=32, mode='test', root=root)

print("len of train_db: ", len(train_db))
print("len of val_db: ", len(val_db))
print("len of test_db: ", len(test_db))

train_loader = DataLoader(train_db, batch_size=batch_size, shuffle=True, num_workers=6)
val_loader = DataLoader(val_db, batch_size=batch_size, num_workers=2)
test_loader = DataLoader(test_db, batch_size=batch_size, num_workers=2)

read csv file: data\images.csv
read csv file: data\images.csv
read csv file: data\images.csv
len of train_db:  359359
len of val_db:  119787
len of test_db:  119787


# gray input

In [3]:
def evaluate_bc(model, loader):
    model.eval()
    sigmoid = nn.Sigmoid()

    correct = 0
    loss = 0
    total = len(loader.dataset)

    for x, y in loader:
        x, y = torch.cat([x,x,x], dim=1).to(device), y.to(device)
        with torch.no_grad():
            logits = model(x)
            pred = (sigmoid(logits) > 0.5).float()
#             print(pred.shape)
#             print(pred)
#             print(torch.unsqueeze(y, dim=1).shape)
#             print(torch.unsqueeze(y, dim=1))
            loss += criterion(logits, torch.unsqueeze(y, dim=1).type_as(logits)).item()*x.size(0)
        correct += torch.eq(pred, torch.unsqueeze(y, dim=1)).sum().float().item()
#         print(correct)

    return correct / total, loss / len(loader.dataset)

In [5]:
viz = visdom.Visdom()
model = Lenet5().to(device)
print(model)
optimizer = optim.Adam(model.parameters(), lr=lr)
criterion = nn.BCEWithLogitsLoss().to(device)

best_acc, best_epoch = 0, 0
global_step = 0
train_loss = 0
losses_perbatch = []
train_losses = []
val_losses = []
val_accs = []
viz.line([0], [-1], win='loss', opts=dict(title='loss'))
viz.line([0], [-1], win='losses', opts=dict(title='loss comparison'))
viz.line([0], [-1], win='val_acc', opts=dict(title='val_acc'))


for epoch in tqdm(range(epochs)):
    
    train_loss = 0
    
    for step, (x, y) in enumerate(train_loader):
        # x: [b, 3, 224, 224] y: [b]
        x, y = torch.cat([x,x,x], dim=1).to(device), y.to(device)

        model.train()
        logits = model(x)
        loss = criterion(logits, torch.unsqueeze(y, dim=1).type_as(logits))

        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        viz.line([loss.item()], [global_step], win='loss', update='append')
        losses_perbatch.append([global_step, loss.item()])
        train_loss += loss.item()*x.size(0)
        global_step += 1

    if epoch % 1 == 0:
        print(f"epoch: {epoch}")

        val_acc, val_loss = evaluate_bc(model, val_loader)
        val_accs.append([epoch, val_acc])
        if val_acc > best_acc:
            best_epoch = epoch
            best_acc = val_acc

            torch.save(model.state_dict(), 'best_letnet5.mdl')

        viz.line([val_acc], [epoch], win='val_acc', update='append')
        viz.line([train_loss/len(train_loader.dataset)], [epoch], win='losses', name='train', update='append')
        viz.line([val_loss], [epoch], win='losses', name='val', update='append')
        
        train_losses.append([epoch, train_loss/len(train_loader.dataset)])
        val_losses.append([epoch, val_loss])
        
print('best acc:', best_acc, 'best epoch:', best_epoch)

pd.DataFrame(losses_perbatch).to_csv('train_loss_perbatch.csv', index=False)
pd.DataFrame(train_losses).to_csv('train_loss.csv', index=False)
pd.DataFrame(val_losses).to_csv('val_loss.csv', index=False)
pd.DataFrame(val_accs).to_csv('val_accs.csv', index=False)

model.load_state_dict(torch.load('best_letnet5.mdl'))
print('loaded from checkpoint')

test_acc, _ = evaluate_bc(model, test_loader)
print('test acc:', test_acc)



Setting up a new session...
  0%|                                                                                           | 0/20 [00:00<?, ?it/s]

Lenet5(
  (model): Sequential(
    (0): Conv2d(3, 6, kernel_size=(5, 5), stride=(1, 1))
    (1): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (2): Conv2d(6, 16, kernel_size=(5, 5), stride=(1, 1))
    (3): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (4): Flatten(start_dim=1, end_dim=-1)
    (5): Linear(in_features=400, out_features=120, bias=True)
    (6): ReLU()
    (7): Linear(in_features=120, out_features=84, bias=True)
    (8): ReLU()
    (9): Linear(in_features=84, out_features=1, bias=True)
  )
)
epoch: 0


  5%|████                                                                            | 1/20 [04:57<1:34:09, 297.33s/it]

epoch: 1


 10%|████████                                                                        | 2/20 [09:52<1:28:50, 296.13s/it]

...

epoch: 18


 95%|███████████████████████████████████████████████████████████████████████████    | 19/20 [1:33:57<04:58, 298.89s/it]

epoch: 19


100%|███████████████████████████████████████████████████████████████████████████████| 20/20 [1:38:57<00:00, 296.89s/it]


best acc: 0.9842887792498352 best epoch: 19
loaded from checkpoint
test acc: 0.9838045864743252


# gray input, lenet with activation 

In [3]:
def evaluate_bc(model, loader):
    model.eval()
    sigmoid = nn.Sigmoid()

    correct = 0
    loss = 0
    total = len(loader.dataset)

    for x, y in loader:
        x, y = torch.cat([x,x,x], dim=1).to(device), y.to(device)
        with torch.no_grad():
            logits = model(x)
            pred = (sigmoid(logits) > 0.5).float()
#             print(pred.shape)
#             print(pred)
#             print(torch.unsqueeze(y, dim=1).shape)
#             print(torch.unsqueeze(y, dim=1))
            loss += criterion(logits, torch.unsqueeze(y, dim=1).type_as(logits)).item()*x.size(0)
        correct += torch.eq(pred, torch.unsqueeze(y, dim=1)).sum().float().item()
#         print(correct)

    return correct / total, loss / len(loader.dataset)

In [4]:
viz = visdom.Visdom()
model = Lenet5_wa().to(device)
print(model)
optimizer = optim.Adam(model.parameters(), lr=lr)
criterion = nn.BCEWithLogitsLoss().to(device)

best_acc, best_epoch = 0, 0
global_step = 0
train_loss = 0
losses_perbatch = []
train_losses = []
val_losses = []
val_accs = []
viz.line([0], [-1], win='loss', opts=dict(title='loss'))
viz.line([0], [-1], win='losses', opts=dict(title='loss comparison'))
viz.line([0], [-1], win='val_acc', opts=dict(title='val_acc'))


for epoch in tqdm(range(epochs)):
    
    train_loss = 0
    
    for step, (x, y) in enumerate(train_loader):
        # x: [b, 3, 224, 224] y: [b]
        x, y = torch.cat([x,x,x], dim=1).to(device), y.to(device)

        model.train()
        logits = model(x)
        loss = criterion(logits, torch.unsqueeze(y, dim=1).type_as(logits))

        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        viz.line([loss.item()], [global_step], win='loss', update='append')
        losses_perbatch.append([global_step, loss.item()])
        train_loss += loss.item()*x.size(0)
        global_step += 1

    if epoch % 1 == 0:
        print(f"epoch: {epoch}")

        val_acc, val_loss = evaluate_bc(model, val_loader)
        val_accs.append([epoch, val_acc])
        if val_acc > best_acc:
            best_epoch = epoch
            best_acc = val_acc

            torch.save(model.state_dict(), 'best_letnet5.mdl')

        viz.line([val_acc], [epoch], win='val_acc', update='append')
        viz.line([train_loss/len(train_loader.dataset)], [epoch], win='losses', name='train', update='append')
        viz.line([val_loss], [epoch], win='losses', name='val', update='append')
        
        train_losses.append([epoch, train_loss/len(train_loader.dataset)])
        val_losses.append([epoch, val_loss])
        
print('best acc:', best_acc, 'best epoch:', best_epoch)

pd.DataFrame(losses_perbatch).to_csv('train_loss_perbatch.csv', index=False)
pd.DataFrame(train_losses).to_csv('train_loss.csv', index=False)
pd.DataFrame(val_losses).to_csv('val_loss.csv', index=False)
pd.DataFrame(val_accs).to_csv('val_accs.csv', index=False)

model.load_state_dict(torch.load('best_letnet5.mdl'))
print('loaded from checkpoint')

test_acc, _ = evaluate_bc(model, test_loader)
print('test acc:', test_acc)



Setting up a new session...
  0%|                                                                                           | 0/20 [00:00<?, ?it/s]

Lenet5(
  (model): Sequential(
    (0): Conv2d(3, 6, kernel_size=(5, 5), stride=(1, 1))
    (1): Tanh()
    (2): AvgPool2d(kernel_size=2, stride=2, padding=0)
    (3): Sigmoid()
    (4): Conv2d(6, 16, kernel_size=(5, 5), stride=(1, 1))
    (5): Tanh()
    (6): AvgPool2d(kernel_size=2, stride=2, padding=0)
    (7): Sigmoid()
    (8): Flatten(start_dim=1, end_dim=-1)
    (9): Linear(in_features=400, out_features=120, bias=True)
    (10): ReLU()
    (11): Linear(in_features=120, out_features=84, bias=True)
    (12): ReLU()
    (13): Linear(in_features=84, out_features=1, bias=True)
  )
)
epoch: 0


  5%|████                                                                            | 1/20 [05:00<1:35:03, 300.20s/it]

epoch: 1


 10%|████████                                                                        | 2/20 [09:53<1:28:51, 296.19s/it]

...

epoch: 18


 95%|███████████████████████████████████████████████████████████████████████████    | 19/20 [1:33:04<04:55, 295.08s/it]

epoch: 19


100%|███████████████████████████████████████████████████████████████████████████████| 20/20 [1:38:03<00:00, 294.20s/it]


best acc: 0.9374389541436049 best epoch: 19
loaded from checkpoint
test acc: 0.9351682569894897


---

# old records backup

## self-implemented LeNet5

In [3]:
def evaluate(model, loader):
    model.eval()

    correct = 0
    total = len(loader.dataset)

    for x, y in loader:
        x, y = x.to(device), y.to(device)
        with torch.no_grad():
            logits = model(x)
            pred = logits.argmax(dim=1)
        correct += torch.eq(pred, y).sum().float().item()

    return correct / total

In [4]:
viz = visdom.Visdom()
model = Lenet5().to(device)
print(model)
optimizer = optim.Adam(model.parameters(), lr=lr)
criterion = nn.CrossEntropyLoss().to(device)

best_acc, best_epoch = 0, 0
global_step = 0
losses = []
val_accs = []
viz.line([0], [-1], win='loss', opts=dict(title='loss'))
viz.line([0], [-1], win='val_acc', opts=dict(title='val_acc'))

for epoch in tqdm(range(epochs)):

    for step, (x, y) in enumerate(train_loader):
        # x: [b, 3, 224, 224] y: [b]
        x, y = x.to(device), y.to(device)

        model.train()
        logits = model(x)
        loss = criterion(logits, y)

        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        losses.append([global_step, loss.item()])
        viz.line([loss.item()], [global_step], win='loss', update='append')
        global_step += 1

    if epoch % 1 == 0:
        print(f"epoch: {epoch}")

        val_acc = evaluate(model, val_loader)
        val_accs.append([epoch, val_acc])
        if val_acc > best_acc:
            best_epoch = epoch
            best_acc = val_acc

            torch.save(model.state_dict(), 'best_letnet5.mdl')

        viz.line([val_acc], [epoch], win='val_acc', update='append')

print('best acc:', best_acc, 'best epoch:', best_epoch)

pd.DataFrame(losses).to_csv('losses.csv', index=True)
pd.DataFrame(val_accs).to_csv('val_accs.csv', index=True)

model.load_state_dict(torch.load('best_letnet5.mdl'))
print('loaded from checkpoint')

test_acc = evaluate(model, test_loader)
print('test acc:', test_acc)



Setting up a new session...
  0%|                                                                                           | 0/20 [00:00<?, ?it/s]

Lenet5(
  (model): Sequential(
    (0): Conv2d(3, 6, kernel_size=(5, 5), stride=(1, 1))
    (1): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (2): Conv2d(6, 16, kernel_size=(5, 5), stride=(1, 1))
    (3): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (4): Flatten(start_dim=1, end_dim=-1)
    (5): Linear(in_features=400, out_features=120, bias=True)
    (6): ReLU()
    (7): Linear(in_features=120, out_features=84, bias=True)
    (8): ReLU()
    (9): Linear(in_features=84, out_features=2, bias=True)
  )
)
epoch: 0


  5%|████                                                                            | 1/20 [05:21<1:41:40, 321.07s/it]

epoch: 1


 10%|████████                                                                        | 2/20 [10:51<1:38:00, 326.67s/it]

...

epoch: 18


 95%|███████████████████████████████████████████████████████████████████████████    | 19/20 [1:43:23<05:19, 319.15s/it]

epoch: 19


100%|███████████████████████████████████████████████████████████████████████████████| 20/20 [1:48:32<00:00, 325.62s/it]


best acc: 0.9922529155918421 best epoch: 18
loaded from checkpoint
test acc: 0.992261263743144


## self-implemented LeNet5 with BCEloss

- for probability output, use BCEWithLogitLoss() is better for binary classifiation.
CrossEntropyLoss is usually for multiclass
Ref:https://discuss.pytorch.org/t/proper-way-of-doing-binary-classification-with-one-probability-output-what-loss-function-activation-function-to-use-and-how-to-compute-accuracy/101895


- for loss calculation, loss.item() is the average loss per batch, so the total loss per batch=loss.item()*x.size(0), and the average loss at the end will be total_loss/len(loader.dataset)
Ref: https://discuss.pytorch.org/t/on-running-loss-and-average-loss/107890


- for accuracy calculation, use 0.5 as the threshold. If sigmoid(logits)>0.5 OR logits>0, pred=1.
Ref: https://discuss.pytorch.org/t/bcewithlogitsloss-and-model-accuracy-calculation/59293


- for plotting two lines (train_loss and val_loss) in the same window of visdom, set up the window as normal, and use 2 lines of viz.line(...win='', name='') in which win specifies the same windows as set up, and different name to differentiate 2 lines 

In [19]:
def evaluate_bc(model, loader):
    model.eval()
    sigmoid = nn.Sigmoid()

    correct = 0
    loss = 0
    total = len(loader.dataset)

    for x, y in loader:
        x, y = x.to(device), y.to(device)
        with torch.no_grad():
            logits = model(x)
            pred = (sigmoid(logits) > 0.5).float()
#             print(pred.shape)
#             print(pred)
#             print(torch.unsqueeze(y, dim=1).shape)
#             print(torch.unsqueeze(y, dim=1))
            loss += criterion(logits, torch.unsqueeze(y, dim=1).type_as(logits)).item()*x.size(0)
        correct += torch.eq(pred, torch.unsqueeze(y, dim=1)).sum().float().item()
#         print(correct)

    return correct / total, loss

In [20]:
viz = visdom.Visdom()
model = Lenet5().to(device)
print(model)
optimizer = optim.Adam(model.parameters(), lr=lr)
criterion = nn.BCEWithLogitsLoss().to(device)

best_acc, best_epoch = 0, 0
global_step = 0
train_loss = 0
losses_perbatch = []
train_losses = []
val_losses = []
val_accs = []
viz.line([0], [-1], win='loss', opts=dict(title='loss'))
viz.line([0], [-1], win='losses', opts=dict(title='loss comparison'))
viz.line([0], [-1], win='val_acc', opts=dict(title='val_acc'))


for epoch in tqdm(range(epochs)):
    
    train_loss = 0
    
    for step, (x, y) in enumerate(train_loader):
        # x: [b, 3, 224, 224] y: [b]
        x, y = x.to(device), y.to(device)

        model.train()
        logits = model(x)
        loss = criterion(logits, torch.unsqueeze(y, dim=1).type_as(logits))

        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        viz.line([loss.item()], [global_step], win='loss', update='append')
        losses_perbatch.append([global_step, loss.item()])
        train_loss += loss.item()*x.size(0)
        global_step += 1

    if epoch % 1 == 0:
        print(f"epoch: {epoch}")

        val_acc, val_loss = evaluate(model, val_loader)
        val_accs.append([epoch, val_acc])
        if val_acc > best_acc:
            best_epoch = epoch
            best_acc = val_acc

            torch.save(model.state_dict(), 'best_letnet5.mdl')

        viz.line([val_acc], [epoch], win='val_acc', update='append')
        viz.line([train_loss/len(train_loader.dataset)], [epoch], win='losses', name='train', update='append')
        viz.line([val_loss], [epoch], win='losses', name='val', update='append')
        
        train_losses.append([epoch, train_loss/len(train_loader.dataset)])
        val_losses.append([epoch, val_loss])
        
print('best acc:', best_acc, 'best epoch:', best_epoch)

pd.DataFrame(losses_perbatch).to_csv('train_loss_perbatch.csv', index=False)
pd.DataFrame(train_losses).to_csv('train_loss.csv', index=False)
pd.DataFrame(val_losses).to_csv('val_loss.csv', index=False)
pd.DataFrame(val_accs).to_csv('val_accs.csv', index=False)

model.load_state_dict(torch.load('best_letnet5.mdl'))
print('loaded from checkpoint')

test_acc, _ = evaluate(model, test_loader)
print('test acc:', test_acc)



Setting up a new session...
  0%|                                                                                           | 0/20 [00:00<?, ?it/s]

Lenet5(
  (model): Sequential(
    (0): Conv2d(3, 6, kernel_size=(5, 5), stride=(1, 1))
    (1): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (2): Conv2d(6, 16, kernel_size=(5, 5), stride=(1, 1))
    (3): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (4): Flatten(start_dim=1, end_dim=-1)
    (5): Linear(in_features=400, out_features=120, bias=True)
    (6): ReLU()
    (7): Linear(in_features=120, out_features=84, bias=True)
    (8): ReLU()
    (9): Linear(in_features=84, out_features=1, bias=True)
  )
)
epoch: 0


  5%|████                                                                            | 1/20 [04:55<1:33:41, 295.84s/it]

epoch: 1


 10%|████████                                                                        | 2/20 [09:49<1:28:18, 294.38s/it]

...

epoch: 18


 95%|███████████████████████████████████████████████████████████████████████████    | 19/20 [1:31:33<04:53, 293.49s/it]

epoch: 19


100%|███████████████████████████████████████████████████████████████████████████████| 20/20 [1:36:37<00:00, 289.85s/it]


best acc: 0.991793767270238 best epoch: 18
loaded from checkpoint
test acc: (0.9916852413033134, 2673.471735427156)


## self-implemented LeNet5 with MSEloss 

In [3]:
def evaluate(model, loader):
    model.eval()
    sigmoid = nn.Sigmoid()

    correct = 0
    loss = 0
    total = len(loader.dataset)

    for x, y in loader:
        x, y = x.to(device), y.to(device)
        with torch.no_grad():
            logits = model(x)
            pred = logits
#             print(pred.shape)
#             print(pred)
#             print(torch.unsqueeze(y, dim=1).shape)
#             print(torch.unsqueeze(y, dim=1))
            loss += criterion(logits, torch.unsqueeze(y, dim=1).type_as(logits)).item()*x.size(0)
#        correct += torch.eq(pred, torch.unsqueeze(y, dim=1)).sum().float().item()
#         print(correct)

    return loss

In [ ]:
viz = visdom.Visdom()
model = Lenet5().to(device)
print(model)
optimizer = optim.Adam(model.parameters(), lr=lr)
criterion = nn.MSELoss().to(device)

best_loss, best_epoch = 0, 0
global_step = 0
train_loss = 0
losses_perbatch = []
train_losses = []
val_losses = []
# val_accs = []
viz.line([0], [-1], win='loss', opts=dict(title='loss'))
viz.line([0], [-1], win='losses', opts=dict(title='loss comparison'))
viz.line([0], [-1], win='val_loss', opts=dict(title='val_loss'))


for epoch in tqdm(range(epochs)):
    
    train_loss = 0
    
    for step, (x, y) in enumerate(train_loader):
        # x: [b, 3, 224, 224] y: [b]
        x, y = x.to(device), y.to(device)

        model.train()
        logits = model(x)
        loss = criterion(logits, torch.unsqueeze(y, dim=1).type_as(logits))

        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        viz.line([loss.item()], [global_step], win='loss', update='append')
        losses_perbatch.append([global_step, loss.item()])
        train_loss += loss.item()*x.size(0)
        global_step += 1

    if epoch % 1 == 0:
        print(f"epoch: {epoch}")

        val_loss = evaluate(model, val_loader)/len(val_loader.dataset)
        # val_accs.append([epoch, val_acc])
        if val_loss > best_loss:
            best_epoch = epoch
            best_loss = val_loss

            torch.save(model.state_dict(), 'best_letnet5.mdl')

        viz.line([val_loss], [epoch], win='val_loss', update='append')
        viz.line([train_loss/len(train_loader.dataset)], [epoch], win='losses', name='train', update='append')
        viz.line([val_loss], [epoch], win='losses', name='val', update='append')
        
        train_losses.append([epoch, train_loss/len(train_loader.dataset)])
        val_losses.append([epoch, val_loss])
        
print('best acc:', best_loss, 'best epoch:', best_epoch)

pd.DataFrame(losses_perbatch).to_csv('train_loss_perbatch.csv', index=False)
pd.DataFrame(train_losses).to_csv('train_loss.csv', index=False)
pd.DataFrame(val_losses).to_csv('val_loss.csv', index=False)
# pd.DataFrame(val_accs).to_csv('val_accs.csv', index=False)

model.load_state_dict(torch.load('best_letnet5.mdl'))
print('loaded from checkpoint')

test_loss = evaluate(model, test_loader)
print('test acc:', test_acc)

